#Input and Model Loading

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch


In [ ]:
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

#Load DialogSum Dataset from Hugging face

In [ ]:
dataset = load_dataset("knkarthick/dialogsum")

# Take a single example
example = dataset["test"][0]
dialogue = example["dialogue"]
reference_summary = example["summary"]


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
print("DIALOGUE:\n", dialogue)
print("\nREFERENCE SUMMARY:\n", reference_summary)


DIALOGUE:
 #Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with t

#Simple Helper Function for Inference

In [ ]:
def generate_summary(input_text, max_length=128):
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


#Without Prompting (Plain Dialogue)

In [ ]:
plain_input = dialogue

print("PLAIN INPUT SAMPLE:\n", plain_input)

PLAIN INPUT SAMPLE:
 #Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please conti

In [ ]:
plain_input = dialogue
plain_summary = generate_summary(plain_input)


print("PLAIN INPUT SUMMARY:\n", plain_summary)

PLAIN INPUT SUMMARY:
 #Person1#: Ms. Dawson, I need you to take a dictation for me.


#Zero-Shot Inference (Instruction Prompting)

In [ ]:
zero_shot_prompt = f"""
Summarize the following conversation:

{dialogue}
"""

zero_shot_summary = generate_summary(zero_shot_prompt)

print("ZERO-SHOT SUMMARY:\n", zero_shot_summary)


ZERO-SHOT SUMMARY:
 #Person1#: Ms. Dawson, I need you to take a dictation for me.


#One-Shot Inference

In [ ]:
one_shot_prompt = f"""
Summarize the conversation.

Conversation:
A: Hi, are you coming to the meeting?
B: Yes, I will be there at 10 AM.
A: Great, see you then.

Summary:
They confirm attending a meeting at 10 AM.

Conversation:
{dialogue}

Summary:
"""

one_shot_summary = generate_summary(one_shot_prompt)

print("ONE-SHOT SUMMARY:\n", one_shot_summary)


ONE-SHOT SUMMARY:
 This memo is to be distributed to all employees by this afternoon.


#Few-Shot Inference (Multiple samples)

In [ ]:
few_shot_prompt = f"""
Summarize the conversation.

Conversation:
A: Do you want coffee?
B: Yes, please.
A: Milk or sugar?
B: Just milk.

Summary:
They discuss preferences for coffee.

Conversation:
A: Are you free this weekend?
B: No, I have an exam.
A: Okay, maybe next week.

Summary:
They postpone plans due to an exam.

Conversation:
{dialogue}

Summary:
"""

few_shot_summary = generate_summary(few_shot_prompt)

print("FEW-SHOT SUMMARY:\n", few_shot_summary)


FEW-SHOT SUMMARY:
 #Person1#: Ms. Dawson, I need you to take a dictation for me.
